In [1]:
import os

In [2]:
samuel_images_test_dir = 'data/samuel_images_test'
samuel_images_train_dir = 'data/samuel_images_train'

In [3]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(400, 600, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [5]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        samuel_images_train_dir,
        target_size=(400, 600),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        samuel_images_test_dir,
        target_size=(400, 600),
        batch_size=20,
        class_mode='binary')

Found 896 images belonging to 2 classes.
Found 383 images belonging to 2 classes.


In [6]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/30
 22/100 [=====>........................] - ETA: 8:31 - loss: 0.7545 - acc: 0.5455

/usr/local/lib/python3.6/site-packages/PIL/Image.py:918: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


100/100 [==============================] - 741s 7s/step - loss: 0.6506 - acc: 0.6274 - val_loss: 0.5375 - val_acc: 0.7588
Epoch 2/30
100/100 [==============================] - 725s 7s/step - loss: 0.4716 - acc: 0.7904 - val_loss: 0.4408 - val_acc: 0.7919
Epoch 3/30
100/100 [==============================] - 719s 7s/step - loss: 0.3228 - acc: 0.8735 - val_loss: 0.5187 - val_acc: 0.7743
Epoch 4/30
100/100 [==============================] - 715s 7s/step - loss: 0.2013 - acc: 0.9319 - val_loss: 0.5233 - val_acc: 0.7774
Epoch 5/30
100/100 [==============================] - 716s 7s/step - loss: 0.1247 - acc: 0.9590 - val_loss: 0.6307 - val_acc: 0.7578
Epoch 6/30
100/100 [==============================] - 714s 7s/step - loss: 0.0607 - acc: 0.9860 - val_loss: 0.7385 - val_acc: 0.7733
Epoch 7/30
100/100 [==============================] - 719s 7s/step - loss: 0.0450 - acc: 0.9890 - val_loss: 0.7147 - val_acc: 0.7733
Epoch 8/30
100/100 [==============================] - 710s 7s/step - loss: 0.046

KeyboardInterrupt: 

In [ ]:
model.save('data/samuel_images_model.h5')